#### Generate Map of Steady State Flow Hydrographs

##### Turn Autosave off
Autosave generates additional work for the version control. Remember to manually save any chances after clearing kernel and output for more effective version control.

In [ ]:
%autosave 0 

##### Google Colab execution
Using this Chrome extension, the github-hosted jupyter notebooks may be opened directly in Google Colaboratory
https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo
more info here:
https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb

The following two StackOverflow posts helped with managing the dependencies
https://stackoverflow.com/questions/53581278/test-if-notebook-is-running-on-google-colab
https://stackoverflow.com/questions/53793731/using-custom-packages-on-google-colaboratory

[![Open This Notebook In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jameshalgren/wrf_hydro_nwm_public/blob/dynamic_channel_routing/trunk/NDHMS/dynamic_channel_routing/notebooks/SteadyNetworkPopupMapDemo.ipynb)

In [1]:
import sys
try:
    import google.colab
    IN_COLAB = True
    !git clone https://github.com/jameshalgren/wrf_hydro_nwm_public.git
    sys.path.append('/content/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src')
except:
    IN_COLAB = False
    sys.path.append(r'../src')

###### Note
9 Sept 2016
The plotting functions seem to require that the cell be executed twice before the output will appear in a standard Jupyter Notebook. 

In [5]:
from __future__ import division
import os
import pandas as pd
from math import ceil
import matplotlib.pyplot as plt
from SteadyNetwork import SteadyNetwork

def main():
    
    root = os.path.dirname(os.path.abspath(''))
    output_folder = os.path.join(root, r'test/', r'output/', r'plot/')
    
    input_type = 'simple'
    input_vars = {}
    input_vars['n_sections'] = 2014
    input_vars['n_timesteps'] = 22
    input_vars['station_downstream'] = 0
    input_vars['station_upstream'] = 1000000
    input_vars['bottom_width_downstream'] = 100
    input_vars['bottom_width_upstream'] = 1000
    input_vars['bottom_z_downstream'] = 0
    input_vars['bottom_z_upstream'] = 100
    input_vars['dx_ds_boundary'] = 1000
    input_vars['S0_ds_boundary'] = 0.0001
    input_vars['manning_n_ds_all'] = 0.035
    input_vars['loss_coeff_all'] = 0.03
    input_vars['hydrograph_steady_time'] = 0
    input_vars['hydrograph_event_width'] = 7
    input_vars['hydrograph_skewness'] = 4
    input_vars['hydrograph_qpeak'] = 5000

    network = SteadyNetwork(input_type = input_type, input_vars = input_vars)

    network.compute_initial_state()
    network.compute_time_steps()

    #Create figure
    fig = plt.figure()
          
    #Plot hydrograph of each section and save it with a name associated to it's COMID
    #TODO: When IDs are matched, COMID should be replaced with whatever ID is associated with lat/lon
    for i, section in enumerate(network.sections):
        a = pd.Series(time_step.depth for i, time_step in enumerate(section.time_steps))
        plt.plot(a)
        plt.savefig(output_folder + 'Hydrograph_' + str(section.comid).zfill(2) + '.jpg')
        plt.close()

if __name__ == "__main__":
    main()

#### Generate map
After writing the image files, generate a map displaying the hydrographs in the locations specified in Locations.csv

In [34]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 30 11:20:56 2019
@author: Camaron.George
"""
#Import necessary modules
import os
import glob
import folium
import base64
import pandas as pd
import geopandas as gpd
from folium import IFrame
from folium import plugins

root = os.path.dirname(os.path.abspath(''))
plot_folder = os.path.join(root, r'test/', r'output/', r'plot/')
text_folder = os.path.join(root, r'test/', r'input/', r'geo/')
map_folder = os.path.join(root, r'test/', r'output/', r'map/')

#Define list of images you want to put in a map popup
files = glob.glob(plot_folder + '*.jpg')

#Read in file with location attributions and define latitude, longitude, and desired ID variables
Locations = gpd.read_file(text_folder + 'Channels/NHD_BrazosLowerColorado_Channels.shp')
Comid = Locations['OBJECTID']
#Comid = Comid[0:len(files)]
Lats = Locations['lat']
#Lats = Lats[0:len(files)]
Lons = Locations['lon']
#Lons = Lons[0:len(files)]
order = Locations['order_']

#Coordinates you want the map to be centered on when it is opened
Center_Coords = (38, -90)

#Create map and popup cluster objects:
m = folium.Map(location = Center_Coords, zoom_start = 5)#, tiles = 'CartoDBPositron')

#Create group of markers you don't want shown when map is initially opened, and add it to the map
notShown = folium.FeatureGroup(name='Lower Order', show=False)
shown = folium.FeatureGroup(name='Higher Order',show=True)
m.add_child(notShown)
m.add_child(shown)

#Create clusters of markers (shown/not shown) and add them to the map (shown) or feature group (not shown)
cluster1 = folium.plugins.MarkerCluster().add_to(notShown)
cluster2 = folium.plugins.MarkerCluster().add_to(shown)

#Allows the user to toggle between each set of markers or show all markers on the map after open
folium.LayerControl().add_to(m)

#Loop through list of images, ind index of lat/lon associated with that image, and add it to the appropriate popoup
for i in range(0,len(files)):
#     ID = os.path.basename(os.path.splitext(files[i])[0])
#     index = Comid.index(ID.split('_')[1])
    encoded = base64.b64encode(open(files[i], 'rb').read())
    html = '<img src="data:image/jpeg;base64,{}">'.format
    iframe = IFrame(html(encoded.decode('UTF-8')), width=500, height=350)
    popup = folium.Popup(iframe)
    #Stream order determines to which cluster each item is assigned
    if order[i] > 5:
        marker = folium.Marker(location=[Lats[i],Lons[i]], popup=popup).add_to(cluster1)#, icon=icon)
    else:
        marker = folium.Marker(location=[Lats[i],Lons[i]], popup=popup).add_to(cluster2)#, icon=icon)
        

#Save map
m.save(map_folder + 'TestMap.html')